<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Mask_NoMask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow
import os
import glob
import pandas as pd
from roboflow import Roboflow
import xml.etree.ElementTree as ET


!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

# Data

In [ ]:
rf = Roboflow(api_key="")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("tensorflow")
os.rename('/content/Mask-Wearing-4', '/content/data')

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mask-Wearing-4 in tensorflow:: 100%|██████████| 154/154 [00:00<00:00, 726.41it/s]


## XML to CSV
Convert annotation files(xml format) to a csv file.
(for manually annotated imges)


In [ ]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text))
            
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
images_path = '/content/'

xml_df = xml_to_csv(images_path)
xml_df.to_csv('/content/annotations.csv', index=None)    

## Label Map

Used in TensorFlow Object Detection API

In [ ]:
labelmap_file = '''item {
    id: 1
    name: 'mask'
}
item {
    id: 2
    name: 'no-mask'
}'''

with open('/content/data/labelmap.pbtxt', 'w+') as f:
  f.write(labelmap_file)
f.close()

## Transfer Dataset to TFrecords

### Install Object Detection API


In [ ]:
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3634, done.
remote: Counting objects: 100% (3634/3634), done.
remote: Compressing objects: 100% (3031/3031), done.
remote: Total 3634 (delta 960), reused 1535 (delta 549), pack-reused 0
Receiving objects: 100% (3634/3634), 47.37 MiB | 28.41 MiB/s, done.
Resolving deltas: 100% (960/960), done.


In [ ]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
import io
import argparse
import collections
from PIL import Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def split(df, group):
	data = collections.namedtuple('data', ['filename', 'object'])
	gb = df.groupby(group)
	return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path, category_idx):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
		encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size

	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(float(row['xmin']) / width)
		xmaxs.append(float(row['xmax']) / width)
		ymins.append(float(row['ymin']) / height)
		ymaxs.append(float(row['ymax']) / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(category_idx[row['class']])

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
	}))
	return tf_example

In [ ]:
for f_name in ['train', 'test', 'valid']:

  img_path = f'/content/data/{f_name}'
  csv_path = f'/content/data/{f_name}/_annotations.csv'
  tfrecord_path = f'/content/data/{f_name}/{f_name}.tfrecord'
  labelmap_path  = '/content/data/labelmap.pbtxt'

  label_map_dict = label_map_util.get_label_map_dict(labelmap_path)
  writer = tf.io.TFRecordWriter(tfrecord_path)

  examples = pd.read_csv(csv_path)
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())

  writer.close()
  print(f'Successfully created the TFRecords: {tfrecord_path}')

Successfully created the TFRecords: /content/data/train/train.tfrecord
Successfully created the TFRecords: /content/data/test/test.tfrecord
Successfully created the TFRecords: /content/data/valid/valid.tfrecord


#Model

## Install a model from the model garden
Choose from a large selection of object detection models in the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [ ]:
MODEL = "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8"

%cd /content/models/research/object_detection
!mkdir pre-trained-models
%cd pre-trained-models
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL}.tar.gz
!tar -xf {MODEL}.tar.gz
!rm {MODEL}.tar.gz

/content/models/research/object_detection
/content/models/research/object_detection/pre-trained-models
--2022-12-30 07:52:44--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.196.128, 2607:f8b0:4001:c1a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.196.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’

faster_rcnn_resnet5 100%[===================>] 202.17M   172MB/s    in 1.2s    

2022-12-30 07:52:45 (172 MB/s) - ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’ saved [211996178/211996178]



## Edit pipeline.config
The pipeline.config file is in protocol buffer format, and we can use the config_util to change the settings we need.

In [ ]:
from object_detection.utils import config_util


from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
pipeline_config = f"./{MODEL}/pipeline.config"
model_dir = f"./{MODEL}/checkpoint"

config = config_util.get_configs_from_pipeline_file(pipeline_config)

model_config = config['model']
model_config.faster_rcnn.num_classes = 2

train_config = config['train_config']

#Starting checkpoin location
train_config.fine_tune_checkpoint = f"/content/models/research/object_detection/pre-trained-models/{MODEL}/checkpoint/ckpt-0"

# GPU memory limits batch size
train_config.batch_size = 32
train_config.use_bfloat16 = False


#...................$$$$$$$$$$$$$$$.................
# some models use "fine_tune" rather than "detection" for fine-tuning
train_config.fine_tune_checkpoint_type = "detection"
# detection      ===> use all the layers as pretrained layers
# classification ===> use only the classification layer as pretrained and initialize the weights randomly


# set auto-augmentation
# del train_config.data_augmentation_options[:]

# a1=train_config.data_augmentation_options.add()
# a1.autoaugment_image.policy_name = "v2"

# training dataset
train_input_config = config['train_input_config']
train_input_config.label_map_path = "/content/data/labelmap.pbtxt"
train_input_config.tf_record_input_reader.input_path[:] = ["/content/data/train/*.tfrecord",
                                                           "/content/data/eval/*.tfrecord"]  # have little data, so we use eval as train data
# evaluation dataset                                                           
eval_input_config = config['eval_input_config']
eval_input_config.label_map_path = "/content/data/labelmap.pbtxt"
eval_input_config.tf_record_input_reader.input_path[:] = ["/content/data/test/*.tfrecord"]

# save pipeline.config file
pipeline_proto = config_util.create_pipeline_proto_from_configs(config)
config_util.save_pipeline_config(pipeline_config=pipeline_proto, directory=f'/content/models/research/object_detection/pre-trained-models/{MODEL}')

Check the contents of the pipeline.config file:


In [ ]:
# !cat /content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config

## Training

In [ ]:
!mkdir /content/data/train_process/

In [ ]:
%cd /content/models/research
!python object_detection/model_main_tf2.py \
    --num_train_steps=1000 \
    --pipeline_config_path=/content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --model_dir=/content/data/train_process/ \
    --alsologtostderr
    #  &> /content/data/train.log

In [ ]:
!tail -n 10 /content/data/train.log

tail: cannot open '/content/data/train.log' for reading: No such file or directory


## Evaluaion

In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --model_dir=/content/data/train_process/ \
    --checkpoint_dir=/content/data/train_process/ \
    --eval_timeout=30 \
    --alsologtostderr

#checkpoint_dir:  Only different to select it as Testing not training the

2022-12-30 07:55:38.241594: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-12-30 07:55:38.241790: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-12-30 07:55:38.241813: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
W1230 07:55:42.792839 140484463544192 model_lib_v2.py:108

In [ ]:
#Copy the piplie.config for later use
!cp /content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config /content/training/.

cp: cannot create regular file '/content/training/.': No such file or directory


In [ ]:
#Export trained model
!python object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=/kaggle/working/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --trained_checkpoint_dir /kaggle/working/training/ \
    --output_directory /kaggle/working/exported-models/effdet_d2

#dd

In [ ]:

#Step 5- Download TensorFlow Model Garden.

#cd into the TensorFlow directory in your Google Drive
%cd '/content/gdrive/My Drive/TensorFlow'

#and clone the TensorFlow Model Garden repository
!git clone https://github.com/tensorflow/models.git

#using a older version of repo (21 Sept 2020)
%cd '/content/gdrive/MyDrive/TensorFlow/models'
!git checkout -f e04dafd04d69053d3733bb91d47d0d95bc2c8199
     

#Step 6- Install some required libraries and tools.

!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim lvis
     

#Step 7- Compile the Protobuf libraries.

#cd into 'TensorFlow/models/research'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.
     

#Step 8- Set the environment.

import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"
sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")
     

#Step 9- Build and Install setup.py.

!python setup.py build
!python setup.py install
     

#Step 10- Test the installation.

#cd into 'TensorFlow/models/research/object_detection/builders/'
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')
     
NOTE:

You should have the images in test and train folder (with their corresponding XML files) and label_map.pbtxt file ready in respective directories.

You should also have the generate_tfrecord.py in your preprocessing directory.

If you don't have these files ready, go back to Step 1 and finish downloading required files.


#Step 11- Generate TFrecords.

#cd into preprocessing directory
%cd '/content/gdrive/My Drive/TensorFlow/scripts/preprocessing'

#run the cell to generate test.record and train.record
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/images/train' -l '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/train.record'
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/images/test' -l '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/test.record'

# !python generate_tfrecord.py -x '[path_to_train_folder]' -l '[path_to_annotations_folder]/label_map.pbtxt' -o '[path_to_annotations_folder]/train.record'
# !python generate_tfrecord.py -x '[path_to_test_folder]' -l '[path_to_annotations_folder]/label_map.pbtxt' -o '[path_to_annotations_folder]/test.record'

     
NOTE:

If you haven't downloaded any pre-trained model yet, go back to Step 1 and finish downloading any pre-trained model of your choice.

We are almost ready to start our model training, just a few more steps before we start our model training!

Step 12- Copying some files

Copy the "model_main_tf2.py" file from "TensorFlow\models\research\object_detection" and paste it into training_demo. We will need this file for training the model.

Copy the "exporter_main_v2.py" file from "TensorFlow\models\research\object_detection" and paste it into training_demo.

We will need this file to export the trained model

Step 13- Configure the pipeline file.

Refer the mentioned Medium article for more details!


#Step 14- Start Tensorboard.

#cd into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'

#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

# %load_ext tensorboard
# %tensorboard --logdir=models/[name_of_pre-trained-model_you_downloaded]
     

#optional
#code to check how much session time is remaining 

import time,psutil
uptime=time.time()-psutil.boot_time()
remaintime=(12*60*60)-uptime
print(remaintime/(60*60))
     

#Step 15- Train the model.

#run the cell to start model training 
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

# !python model_main_tf2.py --model_dir=models/[name_of_pre-trained-model_you_downloaded] --pipeline_config_path=models/[name_of_pre-trained-model_you_downloaded]/pipeline.config
     
Congratulations! You have finished model training!


#Step 16- Export the Trained Model.

#run the cell to start model training 
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/my_model

# !python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/[name_of_pre-trained-model you downloaded]/pipeline.config --trained_checkpoint_dir ./models/[name_of_pre-trained-model_you_downloaded]/ --output_directory ./exported-models/my_model
     
We have finished training and exporting our model. It's time to test our model!


#Step 17- Test the Model.

#Loading the saved_model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/my_model/saved_model"

print('Loading model...', end='')

# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')
     

#Step 18- Testing the Model.

#Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt",use_display_name=True)

#category_index=label_map_util.create_category_index_from_labelmap([path_to_label_map],use_display_name=True)
     

#Step 19- Testing the Model.

#Loading the image
img=['/content/img1.jpg','/content/img2.jpg']
print(img)

#list containing paths of all the images
     

#Step 20- Running the Inference.

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

for image_path in img:

    print('Running inference for {}... '.format(image_path), end='')
    image_np=load_image_into_numpy_array(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()
    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections=detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=1,     #max number of bounding boxes in the image
          min_score_thresh=.3,      #min prediction threshold
          agnostic_mode=False)
    %matplotlib inline
    plt.figure()
    plt.imshow(image_np_with_detections)
    print('Done')
    plt.show()
     